In [0]:
import torch

In [0]:
#Kaggle
filepath = "/content/frames/train_frames"
file_path_train = "/Train.csv"
root_dir_train = "/"

#file_path_test = "/content/96714c94-6-Dataset/Dataset/Test.csv"
#root_dir_test = "/content/frames/test_frames"
#PC
#filepath = "C:\\Users\\ACER\\Desktop\\EmotionDetection\\Dataset\\frames\\train_frames"
#file_path = pd.read_csv('C:\\Users\\ACER\\Desktop\\EmotionDetection\\Dataset\\Train.csv')
#root_dir =  "'C:\\Users\\ACER\\Desktop\\EmotionDetection\\Dataset\\frames\\train_frames"

In [0]:
import glob
image_list_train = glob.glob('/*.jpg')


In [0]:
encoder = {
    'happy': 0,
    'surprised' : 1,
    'angry': 2,
    'sad': 3,
    'Unknown': 4

}
decoder = {
    0 : 'happy',
    1 : 'surprised' ,
    2 : 'angry',
    3 : 'sad',
    4 : 'Unknown'

}

In [0]:
import albumentations as A
import torch 
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from skimage import io, transform
import pandas as pd
import matplotlib.pyplot as plt
from PIL import *
import cv2
import numpy as np

class classDataset(Dataset):
    def __init__(self, csv_file, root_dir, encoder, image_list):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.images_file = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.encoder = encoder
        self.image_list = image_list
        self.aug = A.Compose([
        A.Resize(500, 500),
        A.CenterCrop(200, 200),
        A.RandomCrop(100, 100),
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=(-90, 90)),
        A.VerticalFlip(p=0.5),
        A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        A.ShiftScaleRotate(
                shift_limit=0.0625,
                scale_limit=0.1,
                rotate_limit=30,
                p=1.0),
        A.IAAPiecewiseAffine(p=1.0),
        A.RandomContrast(p=1.0),
        A.Cutout(num_holes=4,  max_h_size=100, max_w_size=20, p=100)
        ])

    def __len__(self):
        return len(self.images_file)

    def __getitem__(self, idx):

        image = plt.imread(self.image_list[idx])
        image = Image.fromarray(image).convert('RGB')
        image = self.aug(image=np.array(image))['image']
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        label =  self.images_file.iloc[idx, 1]
        return torch.tensor(image, dtype=torch.float), self.encoder[label]
  
       
       
train_files = classDataset(file_path_train, root_dir_train, encoder,image_list_train)
train_loader = DataLoader(train_files, shuffle = True, batch_size =  128)


In [0]:
epoch = 5
for i in range ( 0, epoch):
  value = train_epoch(model, i, train_loader)

Training Epoch 0
[Epoch 0] [Batch 2 / 3] [Time 2.61 6.15] [Data 2.58 6.10] [Loss: 1.524538, Acc: 38.93%]Training Epoch 1
[Epoch 1] [Batch 2 / 3] [Time 2.57 6.05] [Data 2.54 6.00] [Loss: 1.562667, Acc: 33.89%]Training Epoch 2
[Epoch 2] [Batch 2 / 3] [Time 2.57 6.09] [Data 2.54 6.04] [Loss: 1.502066, Acc: 37.92%]Training Epoch 3
[Epoch 3] [Batch 2 / 3] [Time 2.68 6.16] [Data 2.65 6.11] [Loss: 1.481296, Acc: 38.59%]Training Epoch 4
[Epoch 4] [Batch 2 / 3] [Time 2.61 6.10] [Data 2.58 6.04] [Loss: 1.522623, Acc: 40.60%]

In [0]:
import numpy as np
def show_img(img):
    plt.figure(figsize=(20,20))
    # unnormalize
    img = img / 2 + 0.5  
    npimg = img.numpy()
    npimg = np.clip(img, 0., 1.)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# show images
data = iter(train_loader)
images, labels = data.next()
from PIL import Image
show_img((images[7]))

In [0]:
from torch import nn
from torchvision import models
class classifie(nn.Module):
    def __init__(self, n_classes, pretrained = True ):
        super(classifie, self).__init__()
        self.cnn_arch = models.resnet18(pretrained = pretrained)
        self.linear1 = nn.Linear(1000,512)
        self.relu = nn.LeakyReLU()
        self.linear2 = nn.Linear(512,128)
        self.softmax = nn.Softmax(dim=1)
        self.linear3 = nn.Linear(128,32)
        self.dropout = nn.Dropout(0.5)
        self.linear4 = nn.Linear(32,n_classes)
   
    def forward(self, input):
        am = self.cnn_arch(input)
        x = self.dropout(self.relu(self.linear1(am)))
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        return x

In [0]:
model = classifie(5).cuda()

In [0]:
import time
import sys
import torch.optim as optim

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def train_epoch(model,epoch,data_loader):
    print('Training Epoch {}'.format(epoch))
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()
    start_time = time.time()
    end_time = time.time()
    t = []
    batch_size= 128
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
    for i, (inputs, targets) in enumerate(data_loader):
        data_time.update(time.time() - end_time)
        if torch.cuda.is_available():
            targets = targets.type(torch.cuda.LongTensor)
            inputs = inputs.cuda()
        outputs = model(inputs)    
        loss = criterion(outputs, targets.type(torch.cuda.LongTensor))
        acc = 100 * (outputs.detach().argmax(1) == targets).cpu().numpy().mean()
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_time.update(time.time() - end_time)
        end_time = time.time()
        sys.stdout.write(
                "\r[Epoch %d] [Batch %d / %d] [Time %.2f %.2f] [Data %.2f %.2f] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    i,
                    len(data_loader),
                    batch_time.val,
                    batch_time.avg,
                    data_time.val,
                    data_time.avg,
                    losses.avg,
                    accuracies.avg))
        
        #print('\nEpoch time {} mins'.format((end_time-start_time)/60))
    
    states = {
        'state_dict': model.state_dict(),
        'optim_dict':optimizer.state_dict()
    }
    return losses.avg, accuracies.avg
   



In [0]:
PATH = '/content/model.pth'
torch.save(model.state_dict(), PATH)
model.load_state_dict(torch.load(PATH))

correct = 0
total = 0
i = 0
with torch.no_grad():
    for data in train_loader:
        images, labels = data[0].to('cuda'), data[1].to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()
        
for i in range( 0 , 41):

    pred_values = predicted[i]
    value1 = pred_values.item()
    label_value = labels[i]
    value2 = label_value.item()
    #print("Predicted : {}        Label: {}".format( decoder[value1], decoder[value2]))

print('Accuracy of the network test images: %d %%' % (
    100 * correct / total))
print(correct, " ", total)

Accuracy of the network test images: 41 %
124   298


In [0]:
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images,labels = images.to('cuda'),labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

classes = ['happy', 'surprised', 'angry', 'sad', 'Unknown']
for i in range(5):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of happy :  0 %
Accuracy of surprised :  0 %
Accuracy of angry : 33 %
Accuracy of   sad :  0 %
Accuracy of Unknown : 100 %


In [0]:
#Testing File
from PIL import Image
import matplotlib.pyplot as plt
file_path_test = "/content/Test.csv"
root_dir_test = "/content"
i = 0
result = []
image_file = []
for i in range(0,186):
  csv_file = pd.read_csv(file_path_test)
  frame = csv_file.iloc[i,0]
  image = root_dir_test  + '/' + frame
  imagetensor = cv2.imread(image)
  #img = Image.open(image)
  #plt.imshow(img)
  to_pil = transforms.ToPILImage()
  images = imagetensor
  fig=plt.figure(figsize=(10,10))
  image = to_pil(images)
  index = predict_image(image)
  image_file.append(frame)
  result.append(decoder[index])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [0]:
test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                     ])
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = image_tensor
    input = input.to('cuda')
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index
  

In [0]:

print('Frame',"\t","Label")
for i in range(0, 186):
  print(image_file[i], "\t", result[i])

In [0]:
dict = {'Frame_ID': image_file, 'Emotion': result}
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv('file2.csv') 
df

,Frame_ID,Emotion
0,test0.jpg,Unknown
1,test1.jpg,Unknown
2,test2.jpg,Unknown
3,test3.jpg,Unknown
4,test4.jpg,Unknown
...,...,...
181,test181.jpg,Unknown
182,test182.jpg,Unknown
183,test183.jpg,Unknown
184,test184.jpg,Unknown
